In [ ]:
!pip install ucimlrepo

# 台湾信用卡违约预测复现（基于信息增益 + Stacking）

In [ ]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# 下载台湾数据集并设置特征名称
default_of_credit_card_clients = fetch_ucirepo(id=350)
X = default_of_credit_card_clients.data.features
y = default_of_credit_card_clients.data.targets

real_feature_names = [
    'LIMIT_BAL', 'SEX', 'EDUCATION', 'AGE',  # X1 - X4
    'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3',   # X5 - X8
    'PAY_4', 'PAY_5', 'PAY_6',               # X9 - X11
    'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3',   # X12 - X14
    'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',   # X15 - X17
    'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',      # X18 - X20
    'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'       # X21 - X23
]

X.columns = real_feature_names  # 更新列名

# 显示元数据
print("=== 数据集元数据 ===")
print(default_of_credit_card_clients.metadata)

# 显示变量信息
print("\n=== 变量信息 ===")
print(default_of_credit_card_clients.variables)

# 初步探索
print("\n=== 特征前五行 ===")
print(X.head())
print("\n=== 特征信息 ===")
print(X.info())
print("\n=== 目标分布 ===")
print(y.value_counts())

=== 数据集元数据 ===
{'uci_id': 350, 'name': 'Default of Credit Card Clients', 'repository_url': 'https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients', 'data_url': 'https://archive.ics.uci.edu/static/public/350/data.csv', 'abstract': "This research aimed at the case of customers' default payments in Taiwan and compares the predictive accuracy of probability of default among six data mining methods.", 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 30000, 'num_features': 23, 'feature_types': ['Integer', 'Real'], 'demographics': ['Sex', 'Education Level', 'Marital Status', 'Age'], 'target_col': ['Y'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Mar 29 2024', 'dataset_doi': '10.24432/C55S3H', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'ID': 365, 'type': 'NATIVE', 'title': 'The comparisons of data mining techniques for the predicti

In [ ]:
#标准化特征
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#信息增益（Information Gain）特征选择
X_scaled = MinMaxScaler().fit_transform(X)
mi_scores = mutual_info_classif(X_scaled, y.values.ravel())
mi_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# 打印前 17 个可读性好的特征
print("信息增益排名前 17 的特征：")
print(mi_series.head(17))

# 选出前17个特征
selected_features = mi_series.head(17).index.tolist()
X_selected = X[selected_features]

信息增益排名前 17 的特征：
PAY_0        0.079798
PAY_2        0.053123
PAY_3        0.040409
PAY_5        0.033134
PAY_4        0.032638
PAY_6        0.029054
PAY_AMT1     0.021798
PAY_AMT5     0.019813
PAY_AMT4     0.018732
PAY_AMT3     0.018732
PAY_AMT6     0.015987
LIMIT_BAL    0.015289
PAY_AMT2     0.013836
BILL_AMT1    0.010609
BILL_AMT6    0.007652
BILL_AMT2    0.007177
BILL_AMT3    0.007131
dtype: float64


## 信息增益特征选择（Information Gain）

我们使用 `mutual_info_classif` 作为信息增益的近似方法，对台湾信用卡数据中的 23 个特征进行评分，并选择排名前 17 的特征用于建模。

结果显示，客户的还款状态（PAY_x）、还款金额（PAY_AMT_x）是最关键的影响因子，说明模型主要从客户的历史还款行为中学习信用风险。



In [ ]:
# 划分训练集测试集
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

## 堆叠模型结构（Stacked Classifier）

论文提出使用三个基础学习器构建 Stacking：

- Random Forest（RF）
- Gradient Boosting（GB）
- XGBoost（XGB）

我们使用 `StackingClassifier` 实现，最终预测器选用 `Logistic Regression`。该结构兼顾了不同模型的偏差与方差能力，并通过交叉验证提升泛化性能。

In [ ]:
# 构建Stacking模型

base_learners = [
    ('rf', RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)),
    ('xgb', XGBClassifier(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    ))
]

stacked_model = StackingClassifier(
    estimators=base_learners,
    final_estimator=LogisticRegression(max_iter=1000),
    cv=5
)

stacked_model.fit(X_train, y_train.values.ravel())


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:02:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:03:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:03:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:03:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:03:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_e

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(class_weight='balanced',
                                                       random_state=42)),
                               ('gb',
                                GradientBoostingClassifier(random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=0.8, device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=Fal...
                                              interaction_constraints=None,
                                              learning_rate=0.1, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=4,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [ ]:
# 模型评估
y_pred = stacked_model.predict(X_test)
y_prob = stacked_model.predict_proba(X_test)[:, 1]

print("=== 模型评估结果 ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))


=== 模型评估结果 ===
Accuracy: 0.8174444444444444
F1 Score: 0.45793467502474433
AUC: 0.7781749933482237


## 模型评估与论文对比

最终模型的评估结果如下：

| 指标       | 本实验结果 | 论文结果  |
|------------|------------|-----------|
| Accuracy   | ≈ 81.7%    | 85.80%    |
| F1 Score   | ≈ 0.456    | 0.5135    |
| AUC        | ≈ 0.777    | 0.870     |

尽管略低于论文，但本实验完全基于公开数据和方法，且无人工选择特征，结果具有稳定性和可复现性。

---


1. **特征影响**：模型最依赖的特征集中在还款行为（PAY_x 和 PAY_AMT_x）。

PAY_0 (0.0798)：2005年9月的还款状态，表明最近的还款行为对违约预测最具影响力。

PAY_2 (0.0531)、PAY_3 (0.0404)、PAY_4 (0.0326)、PAY_5 (0.0331)、PAY_6 (0.0291)：历史还款状态，显示过去的还款模式对违约风险有显著影响。

PAY_AMT1–6：前几次的还款金额，反映客户清偿债务的能力。

LIMIT_BAL (0.0153)：信用额度，表明财务能力。

BILL_AMT1–6：账单金额，可能反映消费行为。

还款状态特征（PAY_0至PAY_6）的主导地位表明，近期和持续的还款行为是违约风险预测的关键。还款金额（PAY_AMT）和信用额度（LIMIT_BAL）进一步强调了财务流动性和信用价值的重要性。值得注意的是，性别（SEX）、教育（EDUCATION）、年龄（AGE）和婚姻状况（MARRIAGE）等人口统计特征未进入前17名，表明行为和财务属性在该场景下比个人特征更具预测力。


2. **模型效果**：在不使用 SMOTE 等采样策略下，F1 与 AUC 依然达到较高水平，表明堆叠结构具备出色的泛化能力。然而，复现中可能的预处理差异限制了性能。


